**Lista de Exercícios N2**

Tema: Grafos e Algoritmos de Busca (Dijkstra, A*, em-ordem, pré-ordem e pós-ordem)

Disciplina: Inteligência Artificial\
Ambiente: Google Colab\
Entrega: via repositório individual no GitHub

**Nome completo: Lucas Costa**\
**Matricula: 2265380**

**Regras Gerais**

- Trabalho individual.

- Linguagem: Python 3 (Google Colab).

- É permitido usar: heapq, numpy, matplotlib, dataclasses.

- Proibido usar funções prontas de shortest path (networkx.shortest_path, scipy.sparse.csgraph.dijkstra, etc.).

O Notebook (.ipynb) deve conter:

- Identificação (nome, turma, link do GitHub)

- Código, testes e reflexões

- Seções organizadas conforme o roteiro abaixo.

**Parte A — Dijkstra (Caminho Mínimo em Grafos Ponderados Positivos)**

> Imagine que você está projetando um sistema de navegação para ambulâncias em uma cidade. Cada interseção é representada como um nó e cada rua como uma aresta ponderada com o tempo médio de deslocamento. Você precisa encontrar a rota mais rápida entre o hospital e o local de atendimento, garantindo que o caminho tenha custo mínimo e seja correto mesmo em grandes redes urbanas.

**Atividades**

- Implemente o algoritmo de Dijkstra, retornando o custo mínimo (dist) e os predecessores (parent).

- Crie uma função reconstruct_path(parent, target) que reconstrua o trajeto.

Teste o algoritmo em um grafo de exemplo.

**Explique (Questões Discursivas):**

1. Por que Dijkstra exige arestas não negativas?

2. Qual a complexidade do algoritmo com lista de adjacência e heapq?

💡 Dica: Compare seu resultado com um mapa simples — se mudar o peso de uma rua, a rota muda?

In [96]:
import heapq

def dijkstra(graph, start):
    dist = {node: float('inf') for node in graph}
    dist[start] = 0
    parent = {node: None for node in graph}
    pq = [(0, start)]

    while pq:
        current_dist, current_node = heapq.heappop(pq)
        if current_dist > dist[current_node]:
            continue
        for neighbor, weight in graph[current_node]:
            distance = current_dist + weight
            if distance < dist[neighbor]:
                dist[neighbor] = distance
                parent[neighbor] = current_node
                heapq.heappush(pq, (distance, neighbor))
    return dist, parent

def reconstruct_path(parent, target):
    path = []
    while target is not None:
        path.append(target)
        target = parent[target]
    path.reverse()
    return path

graph = {
    'Minha Casa': [('A', 12), ('B', 2)],
    'A': [('C', 5), ('D', 14)],
    'B': [('A', 1), ('D', 7)],
    'C': [('D', 4), ('Faculdade', 10)],
    'D': [('Faculdade', 21)],
    'Faculdade': []
}

dist, parent = dijkstra(graph, 'Minha Casa')
path = reconstruct_path(parent, 'Faculdade')

print("Custo mínimo até a Faculdadede:", dist['Faculdade'], "minutos")
print("Rota encontrada:", " → ".join(path))


Custo mínimo até a Faculdadede: 18 minutos
Rota encontrada: Minha Casa → B → A → C → Faculdade


********Por que Dijkstra exige arestas não negativas?

Resp: Se existirem arestas negativas pode acontecer de aparecer um caminho mais curto depois que o nó já foi visitado onde o Dijkstra não corrige a distância resultando em um erro.

!!!!!!!!!!!!!!!!!!!
Qual a complexidade do algoritmo com lista de adjacência e heapq?

Resp: A complexidade de tempo de um algoritmo que utiliza lista de adjacência e fila de prioridade depende do algoritmo aplicado ao grafo e essa combinação é usada em algoritmos de caminho mínimo ou árvore geradora mínima oferecendo uma execução mais eficiente, especialmente em grafos esparsos, onde esta relativamente com poucas arestas.


**Parte B — A-Star (Busca Informada com Heurística Admissível)**

> Agora, considere um robô autônomo que deve se deslocar por um labirinto 2D, evitando obstáculos e chegando ao destino no menor tempo possível.
Diferente do Dijkstra, o robô pode usar uma heurística (como a distância ao alvo) para priorizar rotas promissoras, economizando tempo de busca.

**Atividades**

1. Gere um grid 20x20 com ~15% de obstáculos aleatórios.

2. Implemente a função heuristic(a, b) (distância Manhattan).

3. Desenvolva o algoritmo a_star(grid, start, goal, h) e teste-o.

**Explique (Questões Discursivas):**

1. A* vs Dijkstra: qual expande menos nós?

2. Por que a heurística Manhattan é admissível nesse caso?

💡 Cenário real: O A* é amplamente usado em robôs aspiradores, drones e jogos. Seu desafio é aplicar o mesmo raciocínio.

In [85]:
import heapq
import numpy as np
import random

def generate_grid(size=20, obstacle_prob=0.15):
    grid = np.zeros((size, size), dtype=int)
    for i in range(size):
        for j in range(size):
            if random.random() < obstacle_prob:
                grid[i][j] = 1
    return grid

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def neighbors(pos, grid):
    x, y = pos
    steps = [(1,0), (-1,0), (0,1), (0,-1)]
    result = []
    for dx, dy in steps:
        nx, ny = x + dx, y + dy
        if 0 <= nx < grid.shape[0] and 0 <= ny < grid.shape[1] and grid[nx][ny] == 0:
            result.append((nx, ny))
    return result

def a_star(grid, start, goal, h):
    pq = [(h(start, goal), 0, start)]
    parent = {start: None}
    g_score = {start: 0}

    while pq:
        f, cost, current = heapq.heappop(pq)
        if current == goal:
            # Reconstrói caminho
            path = []
            while current:
                path.append(current)
                current = parent[current]
            path.reverse()
            return path
        for neighbor in neighbors(current, grid):
            tentative_g = g_score[current] + 1
            if neighbor not in g_score or tentative_g < g_score[neighbor]:
                g_score[neighbor] = tentative_g
                f_score = tentative_g + h(neighbor, goal)
                parent[neighbor] = current
                heapq.heappush(pq, (f_score, tentative_g, neighbor))
    return None

grid = generate_grid()
start, goal = (0, 0), (19, 19)
path = a_star(grid, start, goal, heuristic)

if path:
    print("Caminho encontrado pelo robô autônomo:")
    print(path)
    display_grid = grid.copy()
    for x, y in path:
        display_grid[x][y] = 2
    print("\nGrid (0 = livre, 1 = obstáculo, 2 = caminho):")
    print(display_grid)
else:
    print("Nenhum caminho encontrado")

Caminho encontrado pelo robô autônomo:
[(0, 0), (0, 1), (1, 1), (1, 2), (1, 3), (2, 3), (2, 4), (2, 5), (2, 6), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 10), (4, 11), (4, 12), (4, 13), (4, 14), (5, 14), (5, 15), (5, 16), (5, 17), (5, 18), (5, 19), (6, 19), (7, 19), (8, 19), (9, 19), (10, 19), (11, 19), (12, 19), (13, 19), (14, 19), (15, 19), (16, 19), (17, 19), (18, 19), (19, 19)]

Grid (0 = livre, 1 = obstáculo, 2 = caminho):
[[2 2 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0]
 [0 2 2 2 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 2 2 2 2 1 0 1 0 0 0 0 1 0 1 0 0 1]
 [0 0 0 0 0 0 2 2 2 2 2 1 1 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 1 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 1 0 2 2 2 2 2 2]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 2]
 [1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2]
 [0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 2]
 [1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 2]
 [0 1 1 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 2]
 [0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 1 2]
 [0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2]
 [0 0 

!!!!!!!!!!!!!!!
A* vs Dijkstra: qual expande menos nós?

Resp. A* espande menos que a Dijkstra para encontrar o caminho mais curto entre dois pontos. Uma das principais razões é que A* é um algoritmo de busca onde se ja tem a informação, enquanto o Dijkstra faz uma busca "cega" e explora em todas as direções até chegar ao destino.

!!!!!!!!!!!!!!!!!!!!!Por que a heurística Manhattan é admissível nesse caso?

Ela busca o melhor meio de buscar de forma otimista em espaços determinados a direções em 2D, tendo a menor custo de movimentos precisos.

**Parte C — Árvores Binárias e Percursos (DFS em-ordem, pré-ordem e pós-ordem)**

> Você está desenvolvendo um sistema de recomendação que organiza produtos em uma árvore binária de busca (BST), conforme o preço. Cada nó é um produto e a travessia da árvore pode ser usada para: 1. Ordenar produtos (em-ordem); 2. Clonar a estrutura (pré-ordem); 3. Calcular totais ou liberar memória (pós-ordem);

**Atividades**

1. Crie uma BST com os valores: [50, 30, 70, 20, 40, 60, 80, 35, 45].

Implemente os percursos:

1. in_order(root)

2. pre_order(root)

3. post_order(root)

Teste se as saídas correspondem às travessias esperadas.

**Explique (Questões Discursivas):**

1. Em que situação cada tipo de percurso é mais indicado?

In [100]:
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def insert(root, value):
    if root is None:
        return Node(value)
    if value < root.value:
        root.left = insert(root.left, value)
    else:
        root.right = insert(root.right, value)
    return root

def in_order(root):
    if root:
        in_order(root.left)
        print(root.value, end=' ')
        in_order(root.right)

def pre_order(root):
    if root:
        print(root.value, end=' ')
        pre_order(root.left)
        pre_order(root.right)

def post_order(root):
    if root:
        post_order(root.left)
        post_order(root.right)
        print(root.value, end=' ')

values = [50, 30, 70, 20, 40, 60, 80, 35, 45]
root = None
for v in values:
    root = insert(root, v)

print("Percurso em-ordem :")
in_order(root)
print("\nPercurso pré-ordem :")
pre_order(root)
print("\nPercurso pós-ordem :")
post_order(root)


Percurso em-ordem :
20 30 35 40 45 50 60 70 80 
Percurso pré-ordem :
50 30 20 40 35 45 70 60 80 
Percurso pós-ordem :
20 35 45 40 30 60 80 70 50 

!!!!!!!!!!!!!!!!!!  Em que situação cada tipo de percurso é mais indicado?

Resp. Em-Ordem: ordena de forma crescente. Exemplo:Gerar relatorio de dados ordenados.
Pre-Ordem: Mantem a hirarquia. Exemplo: é utilizado em sites de produtos variados mantendo a ordem de cada setor
Pós-Ordem:Mantem a estrutura original da árvore.Exemplo:Soma todos os preços de produtos na árvore.


**Parte D — Reflexões (Respostas Curtas)**

Responda de forma argumentativa (5–10 linhas cada):

1. Quando não é vantajoso usar A*, mesmo tendo uma heurística?

R:A* pode não ser vantajoso quando o custo de calcular a heurística é alto ou quando o ambiente é muito pequeno ou simples.

2. Diferencie corretude e otimalidade nos algoritmos estudados.

R:Corretude: significa que o algoritmo sempre produz uma solução segura que respeita as regras do problema.

Otimslidade:É quando esta certa a solução encontrada de acordo com o criterio de custo minimo.

3. Dê um exemplo do mundo real onde cada tipo de percurso (em, pré, pós) é essencial.

R:
Em:Em sistema de produtos listados em ordem crescente

Pre:Ao Salvar a estrutura de pastas de um sistema

Pós:Calcular totais de estoque em forma de Hieraquia.

4. Como heurísticas inconsistentes podem afetar o resultado do A*?

R:Pode fazer com que a A* reviste nós ja expandidos assim reduzindo a eficiencia.

💬 Sugestão: use exemplos de mapas, jogos, sistemas de busca ou árvores sintáticas.

**Entrega**

Crie um repositório público chamado ia-grafos-seu-nome.

Envie para o repositório o arquivo ia_grafos_buscas.ipynb.

Submeta o link do repositório no ambiente da disciplina **Portal Digital Fametro.**.

**Integridade Acadêmica**

1. O trabalho é individual.
2. Discussões conceituais são permitidas, mas o código deve ser inteiramente autoral.
3. Verificações de similaridade serão aplicadas a todas as submissões.
4. Busque e estude os algoritmos através de pesquisas na internet, livros, slides da disciplina.